In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

In [80]:
df=pd.read_excel(r"C:\Users\Dell\Downloads\Final Participant Data Folder\Final_Train.xlsx")

In [81]:
df.head()

,Qualification,Experience,Rating,Place,Profile,Miscellaneous_Info,Fees
0,"BHMS, MD - Homeopathy",24 years experience,100%,"Kakkanad, Ernakulam",Homeopath,"100% 16 Feedback Kakkanad, Ernakulam",100
1,"BAMS, MD - Ayurveda Medicine",12 years experience,98%,"Whitefield, Bangalore",Ayurveda,"98% 76 Feedback Whitefield, Bangalore",350
2,"MBBS, MS - Otorhinolaryngology",9 years experience,NaN,"Mathikere - BEL, Bangalore",ENT Specialist,NaN,300
3,"BSc - Zoology, BAMS",12 years experience,NaN,"Bannerghatta Road, Bangalore",Ayurveda,"Bannerghatta Road, Bangalore ₹250 Available on...",250
4,BAMS,20 years experience,100%,"Keelkattalai, Chennai",Ayurveda,"100% 4 Feedback Keelkattalai, Chennai",250


In [84]:
df.describe()

,Fees
count,5961.000000
mean,307.944640
std,190.920373
min,5.000000
25%,150.000000
50%,300.000000
75%,500.000000
max,950.000000


In [85]:
df['City'] = df['Place'].str.split(',').str[1]
df['Locality'] = df['Place'].str.split(',').str[0]

In [86]:
df['Qual_1'] = df['Qualification'].str.split(',').str[0]
df['Qual_2'] = df['Qualification'].str.split(',').str[1]
df['Qual_3'] = df['Qualification'].str.split(',').str[2]

In [87]:
df['Misc'] = df['Miscellaneous_Info'].str.split('%').str[0]

df['Misc_len'] = df['Misc'].str.len()

df.loc[df['Misc_len']>3, 'Misc'] = 0

df['Misc'].fillna(0,inplace = True)

df['Misc'] = df['Misc'].astype(int)

df['Misc_2'] = df['Miscellaneous_Info'].str.split('% ').str[1]

df['Misc_3'] = df['Misc_2'].str.split(' ').str[0]

df['Misc_3'].fillna(0,inplace = True)

df['Misc_3_len'] = df['Misc_3'].str.len()

df.loc[df['Misc_3_len']>3, 'Misc_3'] = 0

df.loc[df['Misc_3']==',', 'Misc_3'] = 0

df['Misc_3'] = df['Misc_3'].astype(int)

df['Fees1'] = df['Fees']

df = df.drop(['Fees'],axis=1)

In [88]:
df['Misc_4'] = df['Misc']*np.log((1+df['Misc_3']))

In [89]:
df['Rating'].fillna('0%',inplace = True)

In [90]:
df['Rating'] = df['Rating'].str.slice(stop=-1).astype(int)

In [91]:
df['City'].unique()

array([' Ernakulam', ' Bangalore', ' Chennai', ' Delhi', ' Hyderabad',
       ' Coimbatore', ' Mumbai', ' Thiruvananthapuram', nan, ' Sector 5'],
      dtype=object)

In [92]:
df['Profile'].unique()

array(['Homeopath', 'Ayurveda', 'ENT Specialist', 'Dentist',
       'General Medicine', 'Dermatologists'], dtype=object)

In [93]:
df.dtypes

Qualification          object
Experience             object
Rating                  int32
Place                  object
Profile                object
Miscellaneous_Info     object
City                   object
Locality               object
Qual_1                 object
Qual_2                 object
Qual_3                 object
Misc                    int32
Misc_len              float64
Misc_2                 object
Misc_3                  int32
Misc_3_len            float64
Fees1                   int64
Misc_4                float64
dtype: object

In [94]:
from sklearn.preprocessing import LabelEncoder

In [95]:
df['City'].fillna("XXX",inplace = True)
df['Locality'].fillna("XXX",inplace = True)
df['Qualification'].fillna("XXX",inplace = True)
df['Profile'].fillna("XXX",inplace = True)
df['Qual_1'].fillna("XXX",inplace = True)
df['Qual_2'].fillna("XXX",inplace = True)
df['Qual_3'].fillna("XXX",inplace = True)

In [96]:
lb_City = LabelEncoder()
df["City_code"] = lb_City.fit_transform(df["City"])

In [97]:
lb_Locality = LabelEncoder()
df["Locality_code"] = lb_Locality.fit_transform(df["Locality"])

In [98]:
lb_qual_1 = LabelEncoder()
df["Qual_1_code"] = lb_Locality.fit_transform(df["Qual_1"])

In [ ]:
x = df.loc[df['Qual_1'] == 'Get inspired by remarkable stories of people like you', 'Qual_1_code']

In [99]:
lb_qual_2 = LabelEncoder()
df["Qual_2_code"] = lb_Locality.fit_transform(df["Qual_2"])

In [100]:
lb_qual_3 = LabelEncoder()
df["Qual_3_code"] = lb_Locality.fit_transform(df["Qual_3"])

In [101]:
lb_Profile = LabelEncoder()
df["Profile_code"] = lb_Locality.fit_transform(df["Profile"])

In [102]:
df.dtypes

Qualification          object
Experience             object
Rating                  int32
Place                  object
Profile                object
Miscellaneous_Info     object
City                   object
Locality               object
Qual_1                 object
Qual_2                 object
Qual_3                 object
Misc                    int32
Misc_len              float64
Misc_2                 object
Misc_3                  int32
Misc_3_len            float64
Fees1                   int64
Misc_4                float64
City_code               int32
Locality_code           int32
Qual_1_code             int32
Qual_2_code             int32
Qual_3_code             int32
Profile_code            int32
dtype: object

In [103]:
np.where(df.dtypes != np.float)[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 16, 18, 19,
       20, 21, 22, 23], dtype=int64)

In [105]:
df_xgb = df[['Qual_1_code','Qual_2_code','Qual_3_code','Experience', 'Rating','Profile_code','Locality_code','City_code','Misc_4','Fees1']]

In [106]:
#df_xgb = df[['Qual_1_code','Qual_3_code','years_exp','Locality_code','Misc_4','Fees1']]

In [110]:
X, y = df_xgb.iloc[:,:-1],df_xgb.iloc[:,-1]

In [111]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [113]:
#data_dmatrix = xgb.DMatrix(data=X,label=y)

In [114]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [115]:
# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, n_estimators = 10)

In [116]:
# xg_reg.fit(X_train,y_train)

# preds = xg_reg.predict(X_test)

In [117]:
#rmse = np.sqrt(mean_squared_error(y_test, preds))
# print("RMSE: %f" % (rmse))